In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)

In [3]:
# Read the Dataset given path to JSON file
# input: JSON file   -> output: list of size 4 (sentence, EXR, TOP, TOP_DECOUPLED) * number of strings
# we will use this to read the Training/ evaluation / test datasets. 
def read_dataset(path: str):
    data = pd.read_json(path, lines = True)
    columns = data.columns.tolist()
    parsed_json = [None]*len(columns)
    for i in range(0,len(columns)):
        parsed_json[i] = data[columns[i]] # IDK will it be easier to us to work with pandas or numpy
    return parsed_json # we store data in list of PD.Series for now

In [17]:
# the function takes SERIES of string sentences -> outputs SERIES of String sentences
def text_normalization(sentences):
# convert words to lower
    sentences = sentences.str.lower()
# sizes are flexible some are party-sized, party size , lunch - sized ....
# i will assume one format A-B : party-sized
# now we build our regex where we search for "(some shit) size" and convert it to some shit-size 
    pattern_size =r'''(?xm) # multiline verbose flag to enable comments
        \b([a-z]*)[-\s]*(size)(?:d)?\b # i search for alphabets that is followed by one or more spaces or "-" then one or more spaces
                                       # then size, not capturing d because i don't want it anymore (i don't want it in group actually)
        '''
    sentences = sentences.str.replace(pattern_size, r"\1-\2",regex=True)
# sometimes they ask the pizza to be gluten-free, gluten free -> normalizing this to gluten-free
    pattern_free = r'''(?xm) 
        \b([a-z]*)[-\s]*(free)\b
        '''
    sentences = sentences.str.replace(pattern_free, r"\1-\2",regex=True)
# sometimes they refer to pizza as pie : WE ONLY SELL PIZZA
    sentences = sentences.str.replace("pie", "pizza")
# there are alot of Quantitative items 3 pies, 250 ml, 552 ....
# we can normalizing this to a quantity flag something like <Q>
    pattern_digits = r"[0-9]+"
# we make the flag as lowercase "quta" short for quantity 
    sentences = sentences.str.replace(pattern_digits, "quta", regex=True)
#TO DO:
# we still didn't normalize the number of food things like a pizza with , one pizza of
# a and one represent same thing, Three , two should be normalized into quta flags 

# we may add more Normalization Techniques or delete some "WHO KNOWS"

    return sentences

In [11]:
# the function takes SERIES of string sentences -> outputs SET of vocab and , SERIES of list of tokens
def tokenization(sentences):
    # merge the whole series int one sentence to make the vocab extracting faster
    all_words = ' '.join(sentences)
    # split on \s
    all_words = all_words.split(r" ")
    # keep the unique 
    vocab = set(all_words)
    # i want to tokenize things like I'd to I , 'd
    # new tokens that will come from I'd, it's ,....
    new_tokens = set()
    for word in vocab:
        temp2 = word.split("'")
        # to make sure the 2nd token has its apostrophe: 'd (it should be two splitted words)
        temp2[-1] = "'"+temp2[-1]
        new_tokens.update(temp2)
    vocab.update(new_tokens)
    # we can use EXPAND flag and convert series to dataframe (but who knows do we need to do that?)
    pattern = r"\b([a-z]*)'([a-z]*)\b"
    sentences = sentences.str.replace(pattern, r"\1 '\2",regex=True)
    sentences = sentences.str.split(" ")
   
    return vocab, sentences

In [12]:
sentences, parsed_tree, structured_sentence, decoupled_structured_sentence = read_dataset("./PIZZA_train.json")

In [24]:
'''
now we talk about the data preprocessing
What we should take into consideration? 
1- Word Normalization
2- Word Tokenization
Why we won't use Sentence segmentation? It's useless, orders are one sentence question no clear punctuation exist
'''
# NORMALIZATION
normalized_sentence = text_normalization(sentences.copy())

In [25]:
# TOKENIZATION
# INITIAL tokenization we may need better implementations
vocab, tokenized_sentence = tokenization(normalized_sentence.copy())


In [28]:
print(len(vocab), tokenized_sentence.head())

560 0          [can, i, have, a, large, bbq, pulled, pork]
1    [large, pizza, with, green, pepper, and, with,...
2           [i, 'd, like, a, large, vegetarian, pizza]
3    [party-size, stuffed, crust, pizza, with, amer...
4        [can, i, have, one, personal-size, artichoke]
Name: train.SRC, dtype: object
